# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up pandas, numpy and and sklearn

In [332]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
from pandas import DataFrame, Series
from sklearn import linear_model
from sklearn.metrics import mean_squared_error 

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float,
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str,
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float,
              'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int,
              'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
train = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
test = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)

test.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0114101516,20140528T000000,310000,3,1.0,1430,19901,1.5,0,0,...,7,1430,0,1927,0,98028,47.7558,-122.229,1780,12697
1,9297300055,20150124T000000,650000,4,3.0,2950,5000,2.0,0,3,...,9,1980,970,1979,0,98126,47.5714,-122.375,2140,4000
2,1202000200,20141103T000000,233000,3,2.0,1710,4697,1.5,0,0,...,6,1710,0,1941,0,98002,47.3048,-122.218,1030,4705
3,8562750320,20141110T000000,580500,3,2.5,2320,3980,2.0,0,0,...,8,2320,0,2003,0,98027,47.5391,-122.070,2580,3980
4,7589200193,20141110T000000,535000,3,1.0,1090,3000,1.5,0,0,...,8,1090,0,1929,0,98117,47.6889,-122.375,1570,5080


# Import useful functions from previous notebook

As in Week 2, we convert the Data Frane into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [3]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1
    features = ['constant'] + features
    relevanteFeatures = data_sframe[features].values
    outputArray = np.array(data_sframe[output])
    return(relevanteFeatures, outputArray)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [232]:
def predict_outcome(feature_matrix, weights):
    return(np.dot(feature_matrix, weights))

def get_residual_sum_of_squares(predictions, output):
    residuals = output - predictions
    residualsSquared = residuals * residuals 
    return(residualsSquared.sum()) 

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [17]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print(X)

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [18]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print (norms)

[  5.  13.  17.]


To normalize, apply element-wise division:

In [19]:
print (X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [97]:
def normalize_features(featureMatrix):
    myNorms = np.linalg.norm(featureMatrix, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
    normalizedFeatures = (featureMatrix / myNorms)
    return (normalizedFeatures, myNorms)

To test the function, run the following:

In [98]:
#features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
myArray = np.array([[3.,6.,9.],[4.,8.,12.]])
norms = np.linalg.norm(myArray, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
features,norms = normalize_features(myArray)
print (features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print (norms)
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [347]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [348]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)


We assign some random set of initial weights and inspect the values of `ro[i]`:

In [349]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [350]:
prediction = predict_outcome(simple_feature_matrix,weights )
print(prediction)
print(output)

[ 0.02675867  0.04339256  0.01990703 ...,  0.02289873  0.03178473
  0.02289873]
[ 221900.  538000.  180000. ...,  402101.  400000.  325000.]


Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [351]:

def calculateRo (output, prediction, weights, featureMatrix, index):
    result = np.sum(np.dot(featureMatrix[:,index], output - prediction + np.dot(weights[index], featureMatrix[:,index])))
    return result

def getNextWeight(ro_i,l1_penalty,i):
    half_l1_penalty = l1_penalty/2.
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -1 * half_l1_penalty:
        new_weight_i = ro_i +  half_l1_penalty
    elif ro_i > half_l1_penalty:
        new_weight_i  = ro_i -  half_l1_penalty
    else:
        new_weight_i = 0.
    return new_weight_i


In [352]:
ro = np.zeros(3)
for i in range(simple_feature_matrix.shape[1]):
    ro[i] = calculateRo(output, prediction, weights,simple_feature_matrix,i) 
print(ro)
roUpperBound = ro *2 
print(roUpperBound)

question1LePenalties  = np.array([1.4e8,1.64e8,1.73e8,1.9e8,2.3e8])
for i in range(len(question1LePenalties)):
    w1Weight =  getNextWeight(ro[1],question1LePenalties[i],1)
    w2Weight =  getNextWeight(ro[2],question1LePenalties[i],2)
    print((question1LePenalties[i], w1Weight,w2Weight))



[ 79400300.01452291  87939470.82325177  80966698.66623946]
[  1.58800600e+08   1.75878942e+08   1.61933397e+08]
(140000000.0, 17939470.823251769, 10966698.666239455)
(164000000.0, 5939470.8232517689, 0.0)
(173000000.0, 1439470.8232517689, 0.0)
(190000000.0, 0.0, 0.0)
(230000000.0, 0.0, 0.0)


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [385]:
(1.64e8,1.73e8)

(164000000.0, 173000000.0)

***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [ ]:
(1.9e8,2.3e8)

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [354]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    prediction = predict_outcome(feature_matrix,weights )
    ro_i = calculateRo(output, prediction, weights,feature_matrix,i)
    next_weight_i = getNextWeight(ro_i,l1_penalty,i)
    return next_weight_i

To test the function, run the following cell:

In [355]:
# should print 0.425558846691
import math
print (lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [356]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    currentWeights = initial_weights
    maxChangeInCoordinate = tolerance + 1 
    while maxChangeInCoordinate >= tolerance :
        oldWeights = currentWeights.copy()
        for i in range(len(initial_weights)):
            currentWeights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, currentWeights, l1_penalty)
        maxChangeInCoordinate = np.abs(oldWeights - currentWeights)
        maxChangeInCoordinate = max (maxChangeInCoordinate)
    return currentWeights
        
        
    

Using the following parameters, learn the weights on the sales dataset. 

In [368]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [369]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [359]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

1. What is the RSS of the learned model on the normalized dataset?


In [370]:
newPrediction = predict_outcome(normalized_simple_feature_matrix,weights )
newRss = get_residual_sum_of_squares(newPrediction, output)
print(newRss)

1.63049247672e+15


2. Which features had weight zero at convergence?

In [371]:
print(weights)

[ 21624997.95951911  63157247.20788954         0.        ]


# Evaluating LASSO fit with more features

Let us consider the following set of features.

In [373]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [374]:
(multi_train_set, multi_train_output) = get_numpy_data(train, all_features, 'price')

(multi_train_set_normalized, multi_train_norms) = normalize_features(multi_train_set)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [378]:
def printWeights(features, weights):
    print("constant", " ", weights[0])
    for i in range(len(features)):
        print(features[i], " ", weights[i+1]) 

initial_weights = np.zeros(multi_train_set_normalized.shape[1])
tolerance=1
l1_penalty_7=1e7
weights1e7 = lasso_cyclical_coordinate_descent(multi_train_set_normalized, multi_train_output,
                                            initial_weights, l1_penalty_7, tolerance)


constant   24429600.2344
bedrooms   0.0
bathrooms   0.0
sqft_living   48389174.7715
sqft_lot   0.0
floors   0.0
waterfront   3317511.21492
view   7329961.81171
condition   0.0
grade   0.0
sqft_above   0.0
sqft_basement   0.0
yr_built   0.0
yr_renovated   0.0


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [379]:
printWeights(all_features, weights1e7)

constant   24429600.2344
bedrooms   0.0
bathrooms   0.0
sqft_living   48389174.7715
sqft_lot   0.0
floors   0.0
waterfront   3317511.21492
view   7329961.81171
condition   0.0
grade   0.0
sqft_above   0.0
sqft_basement   0.0
yr_built   0.0
yr_renovated   0.0


Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [383]:
l1_penalty_8=1e8
initial_weights = np.zeros(multi_train_set_normalized.shape[1])
weights1e8 = lasso_cyclical_coordinate_descent(multi_train_set_normalized, multi_train_output,
                                            initial_weights, l1_penalty_8, tolerance)

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [384]:
printWeights(all_features, weights1e8)

constant   71114625.7149
bedrooms   0.0
bathrooms   0.0
sqft_living   0.0
sqft_lot   0.0
floors   0.0
waterfront   0.0
view   0.0
condition   0.0
grade   0.0
sqft_above   0.0
sqft_basement   0.0
yr_built   0.0
yr_renovated   0.0


Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [381]:
l1_penalty_4=1e4
tolerance_4=5e5
initial_weights = np.zeros(multi_train_set_normalized.shape[1])
weights1e4 = lasso_cyclical_coordinate_descent(multi_train_set_normalized, multi_train_output,
                                            initial_weights, l1_penalty_4, tolerance_4)

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [382]:
printWeights(all_features, weights1e4)

constant   78564738.3416
bedrooms   -22097398.9243
bathrooms   12791071.8728
sqft_living   93808088.0928
sqft_lot   -2013172.75705
floors   -4219184.93265
waterfront   6482842.81754
view   7127408.53481
condition   5001664.8547
grade   14327518.4371
sqft_above   -15770959.1524
sqft_basement   -5159591.22213
yr_built   -84495341.7684
yr_renovated   2824439.49704


## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [326]:
print(weights1e8)
print(weights1e7)
print(weights1e4)

normalized_weights1e4 = weights1e4 / multi_train_norms
normalized_weights1e7 = weights1e7 / multi_train_norms
normalized_weights1e8 = weights1e8 / multi_train_norms

[ 71114625.71488702         0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.        ]
[ 24429600.23440311         0.                 0.          48389174.77154896
         0.                 0.           3317511.21492165
   7329961.81171426         0.                 0.                 0.
         0.                 0.                 0.        ]
[ 78564738.34156756 -22097398.92430529  12791071.87278517  93808088.092812
  -2013172.75704955  -4219184.93265017   6482842.81753506
   7127408.53480688   5001664.85469642  14327518.43714053 -15770959.152374
  -5159591.22213149 -84495341.76843643   2824439.49703684]


In [327]:
print(normalized_weights1e7[3])

161.317457646


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [328]:
(test_feature_matrix, test_output) = get_numpy_data(test, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [329]:
prediction_4 = predict_outcome(test_feature_matrix,normalized_weights1e4 )
rss_4 = get_residual_sum_of_squares(prediction_4, test_output)
print(rss_4)

2.28459958971e+14


In [330]:
prediction_7 = predict_outcome(test_feature_matrix,normalized_weights1e7 )
rss_7 = get_residual_sum_of_squares(prediction_7, test_output)
print(rss_7)

2.7596207592e+14


In [331]:
prediction_8 = predict_outcome(test_feature_matrix,normalized_weights1e8 )
rss_8 = get_residual_sum_of_squares(prediction_8, test_output)
print(rss_8)

5.37166151497e+14


***QUIZ QUESTION***

Which model performed best on the test data?